In [1]:
import numpy as np
import pandas as pd

In [2]:
import gzip
import os
import gc

In [3]:
from keras.models import Sequential, load_model
from keras.layers import Conv1D, GlobalMaxPool1D, Dense, Dropout, Activation, Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [4]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

In [5]:
from gensim.models import Word2Vec, KeyedVectors

C:\Users\k\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [6]:
hyperparam = {'sequence_len': 100,
              'embedding_dim': 300, 
              'filters': 200, 
              'kernel_size': 3,
              'dropout' : 0.5,
              'dense_units': 100,
              'batch_size': 512,
              'epochs': 1000,
              'steps_per_epochs': None,
              'early_stopping': True,
              'vocab_size': None,
              'learning_rate' : 0.0005,
              'gradient_clip_value' : None,
              'gradient_clip_norm' : None,
              'validation_split': 0.1,
              'missing_word_vectors': 'normal',
              'conv_activation': 'relu', 
              'dense_activation':'relu',
              'n_class': 6}

In [7]:
if hyperparam['early_stopping']:
    hyperparam['validation_split'] = max(0.1, hyperparam['validation_split'])

In [8]:
name = '_'.join(['CNN_2_FC_Missing_relu', 
                 str(hyperparam['sequence_len']), 
                 str(hyperparam['filters']), 
                 str(hyperparam['kernel_size']), 
                 str(int(hyperparam['dropout']*100))])

In [9]:
save_predictions = True
save_model = False
use_best_checkpoint = True

In [10]:
try:
    word_vec
except NameError:
    if os.path.exists('./data/GoogleNews-vectors-negative300.bin'):
        word_vec = KeyedVectors.load_word2vec_format(fname='./data/GoogleNews-vectors-negative300.bin', binary=True)
    elif os.path.exists('./data/GoogleNews-vectors-negative300.bin.gz'):
        google_w2v = gzip.open('./data/GoogleNews-vectors-negative300.bin.gz', 'rb')
        word_vec = KeyedVectors.load_word2vec_format(fname=google_w2v, binary=True)
        del google_w2v
    else:
        print('Embedings not found')

In [11]:
tokenizer = Tokenizer(num_words=hyperparam['vocab_size'], filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'')

In [12]:
train = pd.read_csv('./data/train.csv')

In [13]:
test = pd.read_csv('./data/test.csv')

In [14]:
train_text = train['comment_text'].astype('str').values

In [15]:
test_text = test['comment_text'].astype('str').values

In [16]:
tokenizer.fit_on_texts(train_text)

In [17]:
train_seq = tokenizer.texts_to_sequences(train_text)

In [18]:
test_seq = tokenizer.texts_to_sequences(test_text)

In [19]:
if not hyperparam['vocab_size']:
    hyperparam['vocab_size'] = len(tokenizer.word_index)
print('Vocab Size:', hyperparam['vocab_size'])

Vocab Size: 193264


In [20]:
if hyperparam['missing_word_vectors']=='normal':
    embed_list = []
    for word, index in tokenizer.word_index.items():
        if index >= hyperparam['vocab_size']: 
            continue
        try:
            embed_list.append(word_vec.wv[word])
        except KeyError:
            pass
    a = np.array(embed_list)
    embedding_matrix = np.array(np.random.normal(a.mean(), a.std(), (hyperparam['vocab_size'], hyperparam['embedding_dim'])), dtype=np.float32)
    del embed_list
    del a
else:
    embedding_matrix = np.zeros((hyperparam['vocab_size'], hyperparam['embedding_dim']), dtype=np.float32)

In [21]:
unknown_count = 0
unknown_freq = {}
for word, index in tokenizer.word_index.items():
    if index >= hyperparam['vocab_size']: 
            continue
    try:
        embedding_matrix[index, :] = word_vec.wv[word]
    except KeyError:
        unknown_freq[word] = tokenizer.word_counts[word]
        unknown_count += 1

In [22]:
print('Unknown words', unknown_count)

Unknown words 127113


In [23]:
print('Unknown Freq', sum(unknown_freq.values()))

Unknown Freq 1450599


In [24]:
X_train = pad_sequences(train_seq, maxlen=hyperparam['sequence_len'], truncating='post', padding='post')

In [25]:
y_train = train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values

In [26]:
if hyperparam['validation_split'] or hyperparam['early_stopping']:
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=hyperparam['validation_split'], random_state=22)

In [27]:
if not os.path.exists(f'./{name}'):
    os.mkdir(f'./{name}')

In [28]:
with open(f'./{name}/Hyperparameters.csv', 'w') as file:
    file.write('{}, {}'.format('Hyperparameter', 'Value'))
    for key in hyperparam.keys():
        file.write('\n {}, {}'.format(str(key), str(hyperparam[key])))

In [29]:
embedding_matrix.shape

(193264, 300)

In [30]:
def mini_batch_generator():
    global X_train, y_train
    while True:
        yield (X_train[:hyperparam['batch_size'], :], y_train[:hyperparam['batch_size'], :])
        X_train = np.roll(X_train, axis=0, shift=-hyperparam['batch_size'])
        y_train = np.roll(y_train, axis=0, shift=-hyperparam['batch_size'])

In [31]:
def computation_graph():
    model  = Sequential()
    model.add(Embedding(hyperparam['vocab_size'], hyperparam['embedding_dim'], weights=[embedding_matrix], name='Embedding_Layer'))
    model.add(Conv1D(filters=hyperparam['filters'], 
                     kernel_size=hyperparam['kernel_size'],
                     activation = hyperparam['conv_activation'],
                     name= '_'.join(['Convolution_1D', str(hyperparam['filters']), str(hyperparam['kernel_size']), str(hyperparam['conv_activation'])])
                    ))
    model.add(GlobalMaxPool1D(name='Global_Max_Pooling'))
    model.add(Dense(units=hyperparam['filters'], name='Dense_'+str(hyperparam['filters'])))
    model.add(Dropout(rate=hyperparam['dropout'], name = 'Dropout_' + str(hyperparam['dropout'])))
#     model.add(Activation(hyperparam['dense_activation'], name='Activation_'+str(hyperparam['dense_activation'])))
    model.add(Dense(units=hyperparam['dense_units'], name='Dense_'+str(hyperparam['dense_units'])))
    model.add(Dropout(rate=hyperparam['dropout'], name = 'Dropout2_' + str(hyperparam['dropout'])))
    model.add(Activation(hyperparam['dense_activation'], name='Activation2_'+str(hyperparam['dense_activation'])))
    model.add(Dense(units=hyperparam['n_class'], activation='sigmoid', name='Dense_'+str(hyperparam['n_class'])+'_Sigmoid'))
    return model

In [32]:
model = computation_graph()

In [33]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Embedding_Layer (Embedding)  (None, None, 300)         57979200  
_________________________________________________________________
Convolution_1D_200_3_relu (C (None, None, 200)         180200    
_________________________________________________________________
Global_Max_Pooling (GlobalMa (None, 200)               0         
_________________________________________________________________
Dense_200 (Dense)            (None, 200)               40200     
_________________________________________________________________
Dropout_0.75 (Dropout)       (None, 200)               0         
_________________________________________________________________
Dense_100 (Dense)            (None, 100)               20100     
_________________________________________________________________
Dropout2_0.75 (Dropout)      (None, 100)               0         
__________

In [34]:
if hyperparam['gradient_clip_norm'] is None and hyperparam['gradient_clip_value'] is None:
    model.compile(loss='binary_crossentropy', 
                  optimizer=Adam(hyperparam['learning_rate']),
                  metrics=['accuracy'])
elif hyperparam['gradient_clip_norm'] is None:
    model.compile(loss='binary_crossentropy', 
                  optimizer=Adam(hyperparam['learning_rate'], 
                  clipvalue=hyperparam['gradient_clip_value']),
                  metrics=['accuracy'])
elif hyperparam['gradient_clip_value'] is None:
    model.compile(loss='binary_crossentropy', 
                  optimizer=Adam(hyperparam['learning_rate'], 
                  clipnorm = hyperparam['gradient_clip_norm']),
                  metrics=['accuracy'])
else:
    model.compile(loss='binary_crossentropy', 
                  optimizer=Adam(hyperparam['learning_rate'], 
                  clipvalue=hyperparam['gradient_clip_value'],
                  clipnorm = hyperparam['gradient_clip_norm']),
                  metrics=['accuracy'])

In [35]:
if hyperparam['early_stopping']:
    callback = [EarlyStopping(verbose=1, patience=3), ModelCheckpoint(f'./{name}/model_best.h5', save_best_only=True)]
    if hyperparam['steps_per_epochs']:
        callback = [EarlyStopping(verbose=1, patience=5), ModelCheckpoint(f'./{name}/model_best.h5', save_best_only=True)]
    validation_data = (X_val, y_val)
elif hyperparam['validation_split']:
    callback = [ModelCheckpoint(f'./{name}/model_best.h5', save_best_only=True)]
    validation_data = (X_val, y_val)
else:
    callback = None
    validation_data = None                                               

In [36]:
if hyperparam['steps_per_epochs']:
        history = model.fit_generator(generator=mini_batch_generator(),
                              epochs=hyperparam['epochs'], 
                              callbacks=callback,
                              validation_data = validation_data, 
                              steps_per_epoch=hyperparam['steps_per_epochs'])
else:
        history = model.fit(x=X_train, y=y_train,
                          validation_data = validation_data,
                          epochs=hyperparam['epochs'],
                          batch_size=hyperparam['batch_size'], 
                          shuffle=True, 
                          callbacks=callback)

Train on 143613 samples, validate on 15958 samples
Epoch 1/1000
143613/143613 [==============================] - 25s - loss: 0.1583 - acc: 0.9451 - val_loss: 0.0574 - val_acc: 0.9800
Epoch 2/1000
143613/143613 [==============================] - 23s - loss: 0.0663 - acc: 0.9775 - val_loss: 0.0523 - val_acc: 0.9805
Epoch 3/1000
143613/143613 [==============================] - 21s - loss: 0.0546 - acc: 0.9808 - val_loss: 0.0526 - val_acc: 0.9811
Epoch 4/1000
143613/143613 [==============================] - 21s - loss: 0.0467 - acc: 0.9830 - val_loss: 0.0564 - val_acc: 0.9810
Epoch 5/1000
143613/143613 [==============================] - 22s - loss: 0.0412 - acc: 0.9844 - val_loss: 0.0513 - val_acc: 0.9814
Epoch 6/1000
143613/143613 [==============================] - 22s - loss: 0.0365 - acc: 0.9860 - val_loss: 0.0550 - val_acc: 0.9815
Epoch 7/1000
143613/143613 [==============================] - 22s - loss: 0.0333 - acc: 0.9870 - val_loss: 0.0575 - val_acc: 0.9816
Epoch 8/1000
143613/14361

In [37]:
if save_predictions:
    X_test = pad_sequences(test_seq, maxlen=hyperparam['sequence_len'], truncating='post', padding='post')
    y_pred = model.predict(X_test, batch_size=hyperparam['batch_size'])
    submission = pd.DataFrame({'id': test['id'], 'toxic': y_pred[:, 0], 'severe_toxic': y_pred[:, 1], 'obscene': y_pred[:, 2], \
                          'threat': y_pred[:, 3], 'insult': y_pred[:, 4], 'identity_hate': y_pred[:, 5]})
    submission.to_csv(f'./{name}/submission.csv', index=False)

In [38]:
if use_best_checkpoint:
    model = load_model(f'./{name}/model_best.h5')
    X_test = pad_sequences(test_seq, maxlen=hyperparam['sequence_len'], truncating='post', padding='post')
    y_pred = model.predict(X_test, batch_size=hyperparam['batch_size'])
    submission = pd.DataFrame({'id': test['id'], 'toxic': y_pred[:, 0], 'severe_toxic': y_pred[:, 1], 'obscene': y_pred[:, 2], \
                          'threat': y_pred[:, 3], 'insult': y_pred[:, 4], 'identity_hate': y_pred[:, 5]})
    submission.to_csv(f'./{name}/submission_checkpoint.csv', index=False)

In [39]:
if save_model:
    model.save(f'./{name}/model_last.h5')
else:
    try:
        os.remove(f'./{name}/model_best.h5')
    except FileNotFoundError:
        pass

In [40]:
#, float_format='%.15f'

In [41]:
# def embed_init(shape, dtype=None):
#     return embedding_matrix

In [42]:
# class CustomInitializer:
#     def __call__(self, shape, dtype=None):
#         return embed_init(shape, dtype=dtype)
# , custom_objects={'embed_init': CustomInitializer}